# Portfolio Optimization using Deep Reinforcement Learning
---

## 4.0 Feature Engineering and Data Preprocessing
---
We perform feature engineering and data preprocessing by:
* Adding Technical Indicators to the data. The technical inicators are used as inputs in the training of our Reinforcement Learning Model
* Adding Coveriance Matrices which is also used as input for training the Models
* Splitting the data into the training set and the testing (trading) set

### 4.1 Import Relevant Libraries

In [1]:
pip install finrl

  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/pip-install-cgunax9y/elegantrl_40b5efc8997f43cd8308bc93839699ed
  Running command git clone --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/pip-install-cgunax9y/elegantrl_40b5efc8997f43cd8308bc93839699ed
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b64df20f552477f24ec50c3e5088c1e241fdc455
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from ta.utils import dropna
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.preprocessor.preprocessors import FeatureEngineer

### 4.2 Load the Data from the csv Files

In [3]:
# Load the whole data set
data = pd.read_csv('./datasets/data.csv')

# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [4]:
pip install pickleshare


Note: you may need to restart the kernel to use updated packages.


In [5]:
%store 

Stored variables and their in-db values:
df_close_full_stocks             ->             date  BRITANNIA.NS  AXISBANK.NS  TATAS
filtered_stocks                  -> Index(['NESTLEIND.NS', 'HDFCBANK.NS', 'HINDUNILVR.


In [6]:
%store -r filtered_stocks

In [7]:
list_of_stocks = filtered_stocks
print(list_of_stocks)

Index(['NESTLEIND.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'TCS.NS',
       'KOTAKBANK.NS', 'HCLTECH.NS', 'WIPRO.NS', 'ASIANPAINT.NS', 'MARUTI.NS',
       'TITAN.NS', 'NTPC.NS', 'BAJAJ-AUTO.NS', 'RELIANCE.NS', 'POWERGRID.NS',
       'LT.NS', 'ICICIBANK.NS', 'GRASIM.NS', 'ITC.NS', 'SBILIFE.NS',
       'HDFCLIFE.NS'],
      dtype='object', name='stock_name')


In [8]:
data.head()

,date,tic,close,high,low,open,volume
0,2018-01-01,POWERGRID.NS,113.484406,111.965652,112.612526,111.965652,6388017
1,2018-01-02,POWERGRID.NS,113.512527,112.106277,112.921906,113.203156,4383705
2,2018-01-03,POWERGRID.NS,113.878151,112.837524,113.428154,113.343781,8187397
3,2018-01-04,POWERGRID.NS,114.103149,112.500031,112.696899,113.625031,4204055
4,2018-01-05,POWERGRID.NS,113.653152,112.837524,113.203156,113.231277,4923559


In [9]:
data = data[data['tic'].isin(list_of_stocks)]

In [10]:
data.tic.unique()

array(['POWERGRID.NS', 'GRASIM.NS', 'HDFCLIFE.NS', 'LT.NS', 'TCS.NS',
       'SBILIFE.NS', 'BAJAJ-AUTO.NS', 'KOTAKBANK.NS', 'ASIANPAINT.NS',
       'WIPRO.NS', 'RELIANCE.NS', 'HINDUNILVR.NS', 'NTPC.NS',
       'HDFCBANK.NS', 'ICICIBANK.NS', 'MARUTI.NS', 'TITAN.NS', 'ITC.NS',
       'HCLTECH.NS', 'NESTLEIND.NS'], dtype=object)

### 4.3 Add Technical Indicators
---
We define a function to add technical indicators to the dataset by making use of the ta library

The following indicators are considered:
* Volatility Average True Range (ATR)
* Volatility Bollinger Band Width (BBW)
* Volume On-balance Volume (OBV)
* Volume Chaikin Money Flow (CMF)
* Trend Moving Average Convergence Divergence (MACD)
* Trend Average Directional Index (ADX)
* Trend Fast Simple Moving Average (SMA)
* Trend Fast Exponential Moving Average (EMA)
* Trend Commodity Channel Index (CCI)
* Momentum Relative Strength Index (RSI)

In [11]:
# Define a Function for adding technical indicators

def add_features(data, feature_list, short_names):
    """
    Function to add technical indicators for features
    -Takes in a dataset with Open, High, Low, Close and Volume
    -Also takes in a list of the technical indicators to be added 
     as well as a list of the shortened indicator names
    """
    
    # list of column names to filter the features
    data_col_names = list(data.columns)
    filter_names = data_col_names + feature_list
    col_rename = data_col_names +  short_names
    
    # Add technical indicators using the ta Library
    data = add_all_ta_features(data, open="open", high="high", 
    low="low", close="close", volume="volume") 
    
    # Filter the Indicators with the required features
    data = data[filter_names]
    data.columns = col_rename # rename the columns to use shortened indicator names
    data = data.dropna()
    
    return data

In [12]:
# List of Features to add
feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf',
               'trend_macd', 'trend_adx', 'trend_sma_fast', 
               'trend_ema_fast', 'trend_cci', 'momentum_rsi']

# Short names of the features
short_names = ['atr', 'bbw','obv','cmf','macd', 'adx', 'sma', 'ema', 'cci', 'rsi']

#feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf','trend_macd']

# Short names of the features
#short_names = ['atr', 'bbw','obv','cmf','macd']

In [13]:
# Add Indicators to our dataset
data_with_features = data.copy()
data_with_features = add_features(data_with_features, feature_list, short_names)

In [14]:
data_with_features.head()

,date,tic,close,high,low,open,volume,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
25,2018-02-06,POWERGRID.NS,110.025024,106.875023,109.659401,106.875023,13495253,2.318797,4.277521,-51293277,-4.375279,-0.846350,0.000000,110.378934,110.548679,-92.730574,38.252399
26,2018-02-07,POWERGRID.NS,110.728149,108.787529,109.968781,110.137527,14316750,2.210667,3.941196,-36976527,-4.328384,-0.769341,0.000000,110.325027,110.576290,-19.305795,44.704442
27,2018-02-08,POWERGRID.NS,109.490654,108.196899,108.675026,109.125031,33536651,2.242725,3.571911,-70513178,-4.345512,-0.798956,25.885701,110.217215,110.409269,-93.095992,37.314365
28,2018-02-09,POWERGRID.NS,109.293777,107.437523,108.675026,108.000031,15183068,2.223766,3.470132,-85696246,-4.014366,-0.828759,24.383058,110.121121,110.237655,-109.780052,36.286633
29,2018-02-12,POWERGRID.NS,111.965652,108.731277,111.656281,109.237526,11747845,2.237640,3.549359,-73948401,-3.821622,-0.629524,23.314385,110.219559,110.503501,90.563033,54.572922


In [15]:
feature_list = list(data_with_features.columns)[7:]

In [16]:
print(feature_list)

['atr', 'bbw', 'obv', 'cmf', 'macd', 'adx', 'sma', 'ema', 'cci', 'rsi']


### 4.4 Add Covariance Matrix
---
We define a function that will add Covarance Matrices to our dataset

In [17]:
def add_cov_matrix(df):
    """
    Function to add Coveriance Matrices as part of the defined states
    """
    # Sort the data and index by date and tic
    df=df.sort_values(['date','tic'],ignore_index=True) 
    df.index = df.date.factorize()[0]
    
    cov_list = [] # create empty list for storing coveriance matrices at each time step
    
    # look back for constructing the coveriance matrix is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values 
        covs = covs#/covs.max()
        cov_list.append(covs)
        
    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    
    return df

In [18]:
# Add Covariance Matrices to our dataset
data_with_features_covs = data_with_features.copy()
data_with_features_covs = add_cov_matrix(data_with_features_covs)

In [19]:
data_with_features_covs.head()

,date,tic,close,high,low,open,volume,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi,cov_list
0,2019-01-09,ASIANPAINT.NS,1414.000000,1397.150024,1402.000000,1402.500000,973687,23.048912,7.896857,5327756272,-4.573096,18.278495,12.349970,1391.958323,1395.415279,99.781902,64.174961,"[[0.00015852269746834745, 1.720325697910041e-0..."
1,2019-01-09,BAJAJ-AUTO.NS,2710.000000,2672.500000,2696.899902,2702.500000,285560,68.528080,9.734268,4727251038,-6.731517,-12.309002,15.475891,2747.995829,2749.349468,-79.336950,41.664892,"[[0.00015852269746834745, 1.720325697910041e-0..."
2,2019-01-09,GRASIM.NS,840.956177,823.226685,831.991821,836.673218,4021049,20.495898,7.643261,2910318411,-8.160890,-5.677552,10.975272,826.057098,829.760860,46.930317,51.833333,"[[0.00015852269746834745, 1.720325697910041e-0..."
3,2019-01-09,HCLTECH.NS,474.274994,466.149994,469.200012,473.500000,2471720,11.073495,4.922655,16292304151,-4.816448,-7.770892,24.752588,478.220833,479.209426,-102.116680,36.874787,"[[0.00015852269746834745, 1.720325697910041e-0..."
4,2019-01-09,HDFCBANK.NS,1060.675049,1051.300049,1058.400024,1059.000000,4284314,14.971774,2.942293,10597644479,-7.130459,6.709806,12.575285,1065.808339,1065.000557,-13.154608,51.853740,"[[0.00015852269746834745, 1.720325697910041e-0..."


In [20]:
# Number of features before adding covariance matrices
num_features_before = len(data_with_features.columns)

# Number of features after adding covariance matrices
num_features_after = len(data_with_features_covs.columns)

# Check if the number of features before and after are the same
print("Number of features before adding covariance matrices:", num_features_before)
print("Number of features after adding covariance matrices:", num_features_after)


Number of features before adding covariance matrices: 17
Number of features after adding covariance matrices: 18


### 4.6 Store the Dataframe

In [21]:
df = data_with_features_covs

In [22]:
%store df

Stored 'df' (DataFrame)


In [23]:
%store

Stored variables and their in-db values:
df                               ->              date            tic        close     
df_close_full_stocks             ->             date  BRITANNIA.NS  AXISBANK.NS  TATAS
filtered_stocks                  -> Index(['NESTLEIND.NS', 'HDFCBANK.NS', 'HINDUNILVR.
